
# Reward metrics

In [ ]:
!pip install yfinance

In [6]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt 
import seaborn as sns 

sns.set()
pd.options.display.float_format = '{:.4f}'.format 

#### Close Values

In [28]:
n_years = 5

In [7]:
def get_close_values(symbols, years):
  end = datetime.now()
  start = end - timedelta(days = 365 * years)
  data = yf.download(symbols,start,end)
  return data.Close.dropna().copy()

In [15]:
symbols = ['VWO','PDBC','SPY','VTWO','ESGU','QQQ','VTI','SCHD','VCLT']
close = get_close_values(symbols,5)
close.describe()

[*********************100%***********************]  9 of 9 completed


,ESGU,PDBC,QQQ,SCHD,SPY,VCLT,VTI,VTWO,VWO
count,1258.0000,1258.0000,1258.0000,1258.0000,1258.0000,1258.0000,1258.0000,1258.0000,1258.0000
mean,69.6529,16.6356,220.0444,55.9520,311.4740,97.4568,160.0391,66.6123,44.2344
std,16.1265,2.1776,78.7002,10.3746,65.9565,8.0293,34.5647,12.7770,5.0141
min,49.1260,11.2200,119.5400,39.5000,222.9500,78.7000,111.9100,40.0300,30.4600
25%,58.2572,15.6300,162.3575,49.0925,265.3475,90.2100,136.3675,58.7837,40.6625
50%,63.7350,16.5900,186.2050,52.5550,288.8950,95.8100,147.7600,62.3125,43.2800
75%,77.3850,17.8250,281.7550,58.3950,340.0750,105.2950,173.4575,68.0525,47.6725
max,108.4600,22.7100,403.9900,80.8600,477.4800,112.9600,242.9600,98.0100,56.4900


In [16]:
def get_return_values(close):
  returns = close.pct_change()
  return returns
  

In [18]:
returns = get_return_values(close)
returns.describe()

,ESGU,PDBC,QQQ,SCHD,SPY,VCLT,VTI,VTWO,VWO
count,1257.0000,1257.0000,1257.0000,1257.0000,1257.0000,1257.0000,1257.0000,1257.0000,1257.0000
mean,0.0007,-0.0001,0.0011,0.0006,0.0007,0.0002,0.0007,0.0005,0.0003
std,0.0122,0.0134,0.0141,0.0120,0.0119,0.0082,0.0121,0.0151,0.0130
min,-0.1200,-0.2656,-0.1198,-0.0995,-0.1094,-0.0882,-0.1138,-0.1347,-0.1209
25%,-0.0027,-0.0050,-0.0040,-0.0036,-0.0030,-0.0031,-0.0032,-0.0059,-0.0058
50%,0.0003,0.0011,0.0017,0.0011,0.0009,0.0003,0.0008,0.0011,0.0011
75%,0.0056,0.0062,0.0077,0.0053,0.0059,0.0036,0.0059,0.0075,0.0073
max,0.0969,0.0433,0.0847,0.0879,0.0906,0.0982,0.0949,0.0963,0.0700


In [19]:
returns.head()

,ESGU,PDBC,QQQ,SCHD,SPY,VCLT,VTI,VTWO,VWO
Date,,,,,,,,,
2017-01-03,nan,nan,nan,nan,nan,nan,nan,nan,nan
2017-01-04,0.0000,0.0017,0.0054,0.0016,0.0059,0.0018,0.0077,0.0150,0.0091
2017-01-05,0.0000,0.0029,0.0057,-0.0009,-0.0008,0.0114,-0.0020,-0.0100,0.0104
2017-01-06,0.0149,0.0000,0.0088,0.0000,0.0036,-0.0069,0.0032,-0.0042,-0.0041
2017-01-09,-0.0038,-0.0151,0.0033,-0.0052,-0.0033,0.0034,-0.0038,-0.0067,-0.0016


## 01. Investment Multiple

Ending values of **1** unit invested

$multiple = \frac{Ending\;Value}{Initial\;Investment}$

*Drawback:*

Does not take into account the investment period. Meaningful only in conjuction with investment period

$multiple = \frac{V_2}{V_1} \cdot \frac{V_3}{V_2} \cdot \frac{V_4}{V_3} \cdot ... \frac{V_n}{V_{n-1}}  = \frac{V_n}{V_1} $

In [34]:
(1 + returns).prod().to_frame()

,0
ESGU,2.2031
PDBC,0.8219
QQQ,3.3491
SCHD,1.8427
SPY,2.1140
VCLT,1.1855
VTI,2.0844
VTWO,1.6546
VWO,1.3710


In [26]:
summary = pd.DataFrame(columns = ['multiple'])
for column in close.columns.values:
  summary.loc[column] = [close[column][-1] / close[column][0]]

summary

,multiple
ESGU,2.2031
PDBC,0.8219
QQQ,3.3491
SCHD,1.8427
SPY,2.1140
VCLT,1.1855
VTI,2.0844
VTWO,1.6546
VWO,1.3710


### 01.01. Price Increase (%)

In [27]:
summary['price_increase_pct'] = (summary.multiple - 1) * 100
summary

,multiple,price_increase_pct
ESGU,2.2031,120.3110
PDBC,0.8219,-17.8114
QQQ,3.3491,234.9088
SCHD,1.8427,84.2694
SPY,2.1140,111.4012
VCLT,1.1855,18.5524
VTI,2.0844,108.4424
VTWO,1.6546,65.4582
VWO,1.3710,37.1026


## 02. Compound Annual Growth Rate (CAGR)

It is an annualized return metric. The constant annual rate of return that would be required for an investment  to grow from its beginning balance to its ending balance, assuming the profits were reinvested at the end of each year of the investment's lifespan

It's used to compare Investments with different Investment Horizons 


$ price \cdot (1 + cagr)^{years} = final\;price$

$cagr = \frac{final\;price}{price}^{\frac{1}{years}} -1$

$cagr = multiple^{\frac{1}{years}} -1$

In [31]:
summary['CAGR'] = summary.multiple ** (1/n_years) - 1
summary

,multiple,price_increase_pct,CAGR
ESGU,2.2031,120.3110,0.1711
PDBC,0.8219,-17.8114,-0.0385
QQQ,3.3491,234.9088,0.2735
SCHD,1.8427,84.2694,0.1300
SPY,2.1140,111.4012,0.1615
VCLT,1.1855,18.5524,0.0346
VTI,2.0844,108.4424,0.1582
VTWO,1.6546,65.4582,0.1060
VWO,1.3710,37.1026,0.0651


## 03. Geometric Mean Return (daily)

Similar to CAGR but using daily data. The constant daily rate of return that would be required for an investment to grow from its beginning balance to its ending balance. assuming the profits were reinvested at the end of each day of the investment's lifespan

$ price \cdot (1 + gmr)^{days} = final\;price$

$gmr = \frac{final\;price}{price}^{\frac{1}{days}} -1$

$gmr = multiple^{\frac{1}{days}} -1$

In [36]:
summary['geo_mean'] = summary.multiple ** (1/(n_years*365)) - 1
summary

,multiple,price_increase_pct,CAGR,geo_mean
ESGU,2.2031,120.3110,0.1711,0.0004
PDBC,0.8219,-17.8114,-0.0385,-0.0001
QQQ,3.3491,234.9088,0.2735,0.0007
SCHD,1.8427,84.2694,0.1300,0.0003
SPY,2.1140,111.4012,0.1615,0.0004
VCLT,1.1855,18.5524,0.0346,0.0001
VTI,2.0844,108.4424,0.1582,0.0004
VTWO,1.6546,65.4582,0.1060,0.0003
VWO,1.3710,37.1026,0.0651,0.0002
